In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from fonte_dados.fabrica import FabricaFonteDados, NERDS_VIAJANTES
from treinamento.treinamento_lda import TreinamentoLda
from repository.mongo_utils import get_pages_content_collection
from repository.wikipedia import WikipediaRepo

In [22]:
import re
from collections import Counter

In [3]:
wikipedia_repo = WikipediaRepo(collection=get_pages_content_collection())
fabrica = FabricaFonteDados()

In [10]:
origem = NERDS_VIAJANTES
num_topics = 100

## Carregando fonte de dados de origem

In [5]:
fonte_dados_origem = fabrica.get_fonte_dados(origem)
fonte_dados_origem.carregar_dados()
documentos_origem = fonte_dados_origem.get_tokens()

### Ajuste de modelo para base de treinamento de origem

In [6]:
treinamento_lda = TreinamentoLda(num_topics=num_topics, passes=2)
resultado_lda = treinamento_lda.ajustar_modelo(documentos_origem)
lda = resultado_lda.modelo_lda

Ajustando modelo com 100 topicos e 2 passes


## Análise de tópicos

### Visualização de top 10 tópicos e suas principais palavras

In [8]:

lda.show_topics(num_topics=10, num_words=20)


[(40,
  '0.016*"cas" + 0.013*"fot" + 0.007*"visit" + 0.007*"ver" + 0.007*"fotograf" + 0.007*"cheg" + 0.006*"vist" + 0.006*"faz" + 0.006*"par" + 0.006*"por" + 0.005*"del" + 0.005*"dia" + 0.005*"fic" + 0.005*"caminh" + 0.005*"local" + 0.005*"não" + 0.005*"ond" + 0.005*"bel" + 0.004*"segu" + 0.004*"cidad"'),
 (12,
  '0.054*"jalap" + 0.019*"rio" + 0.019*"acamp" + 0.014*"passei" + 0.010*"cano" + 0.009*"cacho" + 0.009*"dur" + 0.008*"hor" + 0.008*"trilh" + 0.007*"pra" + 0.007*"algum" + 0.007*"ferv" + 0.006*"poi" + 0.006*"bem" + 0.006*"lei" + 0.006*"tom" + 0.006*"nov" + 0.006*"barrac" + 0.006*"banh" + 0.006*"rem"'),
 (88,
  '0.014*"cervej" + 0.009*"fot" + 0.007*"visit" + 0.007*"ond" + 0.006*"parqu" + 0.005*"jalap" + 0.005*"par" + 0.005*"algum" + 0.004*"viag" + 0.004*"muit" + 0.004*"local" + 0.004*"espaç" + 0.004*"aind" + 0.004*"pass" + 0.004*"fotograf" + 0.004*"bel" + 0.004*"outr" + 0.004*"pod" + 0.004*"ser" + 0.003*"interess"'),
 (3,
  '0.016*"fot" + 0.011*"trilh" + 0.010*"fotograf" + 0.009*"

### Visualizar um tópico individualmente

In [9]:
lda.show_topic(topicid=47, topn=20)

[('vasso', 0.010624579),
 ('preg', 0.009564903),
 ('fot', 0.009415722),
 ('fic', 0.009386034),
 ('jalap', 0.008821247),
 ('macac', 0.008375668),
 ('mir', 0.008362633),
 ('par', 0.0075246436),
 ('visit', 0.007274737),
 ('cacho', 0.0064701424),
 ('algum', 0.006308056),
 ('outr', 0.0059783105),
 ('conhec', 0.0053330287),
 ('lad', 0.0047735428),
 ('rio', 0.004764907),
 ('bem', 0.004597163),
 ('pass', 0.0044109984),
 ('caminh', 0.0043976326),
 ('temp', 0.0043715695),
 ('faz', 0.0042403652)]

## Análise das palavras que mais contribuem para os tópicos

In [38]:
def extrair_palavras(descricao_palavra):
    return re.findall('\".+?\"', descricao_palavra)

In [44]:
"""
Faz analise das top n palavras que mais contribuem para as definicoes dos topics. Se o parametros topn nao eh informado apenas a palavras que mais contribue sera 
considerada. Se for informado 3, por exemplo, as 3 palavras que mais contribuem para um topico serao consideradas.

Retorno: tupla de 2 elementos onde o primeiro eh a lista de topicos com as topn palavras que mais contribuem e o segundo elemento eh um objeto counter que 
mostra as palavras que mais contribuem entre as topn dos topicos.
"""
def analisar_top_n_palavras_em_topicos(lda, num_topics, topn=1):
    topicos = lda.show_topics(num_topics=num_topics, num_words=topn)
    palavras_topicos = [extrair_palavras(p[1]) for p in topicos]
    topn_palavras = [palavra for palavras_topico in palavras_topicos for palavra in palavras_topico]
    print(f'len of top n palavras = {len(topn_palavras)}')
    counter_topn_palavras = Counter(topn_palavras)
    return (topicos, counter_topn_palavras.most_common())

### Análise da primeira palavra que mais contribui para cada tópico

In [47]:
analise_top1 = analisar_top_n_palavras_em_topicos(lda, num_topics=num_topics, topn=1)

len of top n palavras = 100


In [48]:
analise_top1[0][:20]

[(0, '0.022*"jalap"'),
 (1, '0.008*"caminh"'),
 (2, '0.018*"centr"'),
 (3, '0.016*"fot"'),
 (4, '0.009*"par"'),
 (5, '0.011*"fot"'),
 (6, '0.028*"fot"'),
 (7, '0.101*"jalap"'),
 (8, '0.009*"parqu"'),
 (9, '0.013*"parqu"'),
 (10, '0.010*"mus"'),
 (11, '0.009*"algum"'),
 (12, '0.054*"jalap"'),
 (13, '0.011*"passei"'),
 (14, '0.024*"alegr"'),
 (15, '0.096*"nev"'),
 (16, '0.014*"viag"'),
 (17, '0.011*"viag"'),
 (18, '0.016*"lan"'),
 (19, '0.011*"apresent"')]

In [49]:
print(analise_top1[1])

[('"fot"', 29), ('"parqu"', 6), ('"dia"', 5), ('"jalap"', 4), ('"algum"', 3), ('"atacam"', 3), ('"hotel"', 3), ('"cervej"', 3), ('"caminh"', 2), ('"mus"', 2), ('"viag"', 2), ('"restaurant"', 2), ('"cas"', 2), ('"pod"', 2), ('"centr"', 1), ('"par"', 1), ('"passei"', 1), ('"alegr"', 1), ('"nev"', 1), ('"lan"', 1), ('"apresent"', 1), ('"trilh"', 1), ('"visit"', 1), ('"urs"', 1), ('"fotograf"', 1), ('"estr"', 1), ('"santiag"', 1), ('"lago"', 1), ('"vasso"', 1), ('"vulc"', 1), ('"cheg"', 1), ('"final"', 1), ('"cidad"', 1), ('"luí"', 1), ('"ônibu"', 1), ('"lag"', 1), ('"glaci"', 1), ('"ond"', 1), ('"sorvet"', 1), ('"islând"', 1), ('"praç"', 1), ('"igrej"', 1), ('"cat"', 1), ('"wanak"', 1), ('"sant"', 1), ('"bariloch"', 1)]


### Análise das 2 palavras que mais contribuem para cada tópico

In [53]:
analise_top2 = analisar_top_n_palavras_em_topicos(lda, num_topics=num_topics, topn=2)

len of top n palavras = 200


In [54]:
analise_top2[0][:20]

[(0, '0.022*"jalap" + 0.015*"sol"'),
 (1, '0.008*"caminh" + 0.008*"parqu"'),
 (2, '0.018*"centr" + 0.017*"cult"'),
 (3, '0.016*"fot" + 0.011*"trilh"'),
 (4, '0.009*"par" + 0.009*"fic"'),
 (5, '0.011*"fot" + 0.008*"parqu"'),
 (6, '0.028*"fot" + 0.016*"vitr"'),
 (7, '0.101*"jalap" + 0.015*"lev"'),
 (8, '0.009*"parqu" + 0.007*"fot"'),
 (9, '0.013*"parqu" + 0.013*"teatr"'),
 (10, '0.010*"mus" + 0.009*"cel"'),
 (11, '0.009*"algum" + 0.009*"dur"'),
 (12, '0.054*"jalap" + 0.019*"rio"'),
 (13, '0.011*"passei" + 0.010*"cheg"'),
 (14, '0.024*"alegr" + 0.017*"encontr"'),
 (15, '0.096*"nev" + 0.089*"vall"'),
 (16, '0.014*"viag" + 0.009*"visit"'),
 (17, '0.011*"viag" + 0.010*"fot"'),
 (18, '0.016*"lan" + 0.013*"fot"'),
 (19, '0.011*"apresent" + 0.010*"bem"')]

In [55]:
print(analise_top2[1])

[('"fot"', 41), ('"parqu"', 9), ('"dia"', 9), ('"fic"', 6), ('"jalap"', 5), ('"caminh"', 5), ('"algum"', 5), ('"passei"', 5), ('"fotograf"', 5), ('"hotel"', 5), ('"trilh"', 4), ('"restaurant"', 4), ('"viag"', 3), ('"visit"', 3), ('"atacam"', 3), ('"sal"', 3), ('"urs"', 3), ('"cervej"', 3), ('"sol"', 2), ('"par"', 2), ('"mus"', 2), ('"rio"', 2), ('"cheg"', 2), ('"bem"', 2), ('"local"', 2), ('"aplic"', 2), ('"lag"', 2), ('"cas"', 2), ('"lago"', 2), ('"pod"', 2), ('"tir"', 2), ('"centr"', 1), ('"cult"', 1), ('"vitr"', 1), ('"lev"', 1), ('"teatr"', 1), ('"cel"', 1), ('"dur"', 1), ('"alegr"', 1), ('"encontr"', 1), ('"nev"', 1), ('"vall"', 1), ('"lan"', 1), ('"apresent"', 1), ('"outr"', 1), ('"ferv"', 1), ('"curitib"', 1), ('"rock"', 1), ('"prai"', 1), ('"estr"', 1), ('"seward"', 1), ('"santiag"', 1), ('"vasso"', 1), ('"preg"', 1), ('"pont"', 1), ('"vulc"', 1), ('"cinz"', 1), ('"histór"', 1), ('"expos"', 1), ('"lent"', 1), ('"final"', 1), ('"min"', 1), ('"escolh"', 1), ('"experiment"', 1), (

### Análise das 5 palavras que mais contribuem para cada tópico

In [56]:
analise_top5 = analisar_top_n_palavras_em_topicos(lda, num_topics=num_topics, topn=5)

len of top n palavras = 500


In [57]:
analise_top5[0][:20]

[(0,
  '0.022*"jalap" + 0.015*"sol" + 0.015*"fot" + 0.014*"viag" + 0.010*"viaj"'),
 (1,
  '0.008*"caminh" + 0.008*"parqu" + 0.007*"fot" + 0.007*"atacam" + 0.006*"pass"'),
 (2,
  '0.018*"centr" + 0.017*"cult" + 0.013*"visit" + 0.012*"cas" + 0.011*"gui"'),
 (3,
  '0.016*"fot" + 0.011*"trilh" + 0.010*"fotograf" + 0.009*"algum" + 0.008*"caminh"'),
 (4, '0.009*"par" + 0.009*"fic" + 0.008*"outr" + 0.007*"bem" + 0.007*"algum"'),
 (5,
  '0.011*"fot" + 0.008*"parqu" + 0.007*"algum" + 0.006*"visit" + 0.006*"cidad"'),
 (6,
  '0.028*"fot" + 0.016*"vitr" + 0.014*"fotograf" + 0.011*"hotel" + 0.009*"resolv"'),
 (7,
  '0.101*"jalap" + 0.015*"lev" + 0.014*"cacho" + 0.013*"trilh" + 0.012*"fot"'),
 (8,
  '0.009*"parqu" + 0.007*"fot" + 0.007*"santiag" + 0.007*"barrac" + 0.006*"fic"'),
 (9,
  '0.013*"parqu" + 0.013*"teatr" + 0.011*"visit" + 0.010*"cidad" + 0.010*"conhec"'),
 (10,
  '0.010*"mus" + 0.009*"cel" + 0.009*"fot" + 0.009*"ano" + 0.009*"particip"'),
 (11, '0.009*"algum" + 0.009*"dur" + 0.009*"fot" 

In [58]:
print(analise_top5[1])

[('"fot"', 56), ('"fic"', 20), ('"visit"', 17), ('"fotograf"', 16), ('"dia"', 15), ('"parqu"', 13), ('"algum"', 13), ('"trilh"', 12), ('"cidad"', 12), ('"par"', 11), ('"passei"', 11), ('"jalap"', 10), ('"caminh"', 10), ('"bem"', 8), ('"restaurant"', 7), ('"viag"', 6), ('"cas"', 6), ('"hotel"', 6), ('"lag"', 6), ('"tir"', 6), ('"atacam"', 5), ('"pass"', 5), ('"outr"', 5), ('"cacho"', 5), ('"pod"', 5), ('"local"', 5), ('"sal"', 5), ('"urs"', 5), ('"cheg"', 4), ('"igrej"', 4), ('"sol"', 3), ('"centr"', 3), ('"santiag"', 3), ('"conhec"', 3), ('"mus"', 3), ('"faz"', 3), ('"estr"', 3), ('"min"', 3), ('"cervej"', 3), ('"viaj"', 2), ('"tod"', 2), ('"rio"', 2), ('"barc"', 2), ('"vall"', 2), ('"fl"', 2), ('"glaci"', 2), ('"aplic"', 2), ('"abert"', 2), ('"com"', 2), ('"lagun"', 2), ('"prai"', 2), ('"por"', 2), ('"milh"', 2), ('"lago"', 2), ('"revel"', 2), ('"pont"', 2), ('"vulc"', 2), ('"expos"', 2), ('"lent"', 2), ('"víde"', 2), ('"muit"', 2), ('"ônibu"', 2), ('"ond"', 2), ('"and"', 2), ('"praç"

### Análise das 20 palavras que mais contribuem para cada tópico

In [59]:
analise_top20 = analisar_top_n_palavras_em_topicos(lda, num_topics=num_topics, topn=20)

len of top n palavras = 2000


In [ ]:
analise_top20[0][:20]

In [61]:
print(analise_top20[1])

[('"fot"', 76), ('"fic"', 66), ('"algum"', 63), ('"par"', 50), ('"faz"', 45), ('"visit"', 44), ('"dia"', 43), ('"pod"', 40), ('"bem"', 38), ('"cidad"', 38), ('"caminh"', 37), ('"cheg"', 35), ('"passei"', 34), ('"pass"', 33), ('"outr"', 30), ('"fotograf"', 29), ('"parqu"', 28), ('"local"', 28), ('"muit"', 26), ('"hor"', 24), ('"temp"', 24), ('"viag"', 22), ('"conhec"', 22), ('"ond"', 21), ('"trilh"', 20), ('"com"', 20), ('"tod"', 19), ('"ver"', 16), ('"restaurant"', 16), ('"jalap"', 15), ('"pont"', 15), ('"tir"', 14), ('"pouc"', 13), ('"lag"', 13), ('"part"', 12), ('"hotel"', 12), ('"sol"', 11), ('"expos"', 11), ('"cacho"', 11), ('"abert"', 11), ('"gost"', 11), ('"viaj"', 10), ('"cas"', 10), ('"vist"', 10), ('"ser"', 10), ('"estr"', 10), ('"centr"', 9), ('"rio"', 9), ('"bonit"', 9), ('"volt"', 9), ('"interess"', 9), ('"águ"', 8), ('"min"', 8), ('"revel"', 8), ('"sal"', 8), ('"atacam"', 7), ('"alt"', 7), ('"cerr"', 7), ('"são"', 7), ('"pra"', 6), ('"santiag"', 6), ('"bel"', 6), ('"cont"'